In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# Predicting Outcomes of Basketball Games

$\texttt{Spencer Ashton}\\$
$\texttt{Trevor Wai}\\$
$\texttt{Carson Watkin}\\$
$\texttt{Zach Joachim}\\$

### December 5, 2023

# **1 Introduction**
Why do we care about this? Allowing players and coaches to focus on drills and metrics that will more likely lead to winning during practice and make adjustments during games.

# **2 Data**

### 2.1 Box scores. 
We got our data by scraping basketballreference.com.  This website contains the box scores from every game sorted by team and season. WHAT IS A BOX SCORE? We had to combine the box scores from every team in a given season into one big dataframe representing each game played in a season. Another thing we considered: do we compare across seasons or just within a given season? We talk about this more LATER.



### 2.2 Data cleaning.
Talking about the cleaning from above. One challenge we ran into was that this approach included each game twice; once from each participating teams' dataset. We employed SOME TACTIC TO DEAL WITH THAT... Below, we show the process of cleaning the data.

In [46]:
team_data = pd.read_csv('./NBA/Utah Jazz/UTA_2018_team_game_log.csv', index_col=1)
team_data.drop('Unnamed: 0', axis=1, inplace=True)
team_data.fillna(value={'H/A': 'vs'}, inplace=True)
team_data.dropna(inplace=True)
team_data.rename(columns={'Tm': 'Team Score', 'Opp.1': 'Opp Score'}, inplace=True)
team_data["FG_cum"] = team_data["FG"].cumsum()
team_data["3P_cum"] = team_data["3P"].cumsum()
team_data["FGA_cum"] = team_data["FGA"].cumsum()
team_data['TOV_cum'] = team_data['TOV'].cumsum()
team_data['FTA_cum'] = team_data['FTA'].cumsum()
team_data['ORB_cum'] = team_data['ORB'].cumsum()
team_data['TRB_cum'] = team_data['TRB'].cumsum()
team_data['FT_cum'] = team_data['FT'].cumsum()
team_data.head()

,Date,H/A,Opp,W/L,Team Score,Opp Score,FG,FGA,FG%,3P,...,TOV,PF,FG_cum,3P_cum,FGA_cum,TOV_cum,FTA_cum,ORB_cum,TRB_cum,FT_cum
G,,,,,,,,,,,,,,,,,,,,,
1.0,2017-10-18,vs,DEN,W,106.0,96.0,41.0,81.0,0.506,9.0,...,15.0,18.0,41.0,9.0,81.0,15.0,16.0,6.0,36.0,15.0
2.0,2017-10-20,@,MIN,L,97.0,100.0,37.0,77.0,0.481,9.0,...,19.0,21.0,78.0,18.0,158.0,34.0,33.0,13.0,74.0,29.0
3.0,2017-10-21,vs,OKC,W,96.0,87.0,38.0,77.0,0.494,9.0,...,18.0,21.0,116.0,27.0,235.0,52.0,47.0,20.0,115.0,40.0
4.0,2017-10-24,@,LAC,L,84.0,102.0,34.0,79.0,0.430,7.0,...,18.0,23.0,150.0,34.0,314.0,70.0,58.0,28.0,156.0,49.0
5.0,2017-10-25,@,PHO,L,88.0,97.0,32.0,77.0,0.416,6.0,...,23.0,19.0,182.0,40.0,391.0,93.0,83.0,35.0,200.0,67.0


In [47]:
opp_data = pd.read_csv('./NBA/Utah Jazz/UTA_2018_opp_game_log.csv', index_col=1)
opp_data.drop('Unnamed: 0', axis=1, inplace=True)
opp_data.fillna(value={'H/A': 'vs'}, inplace=True)
opp_data.dropna(inplace=True)
opp_data.rename(columns={'Tm': 'Team Score', 'Opp.1': 'Opp Score'}, inplace=True)
opp_data["FG_cum"] = opp_data["FG"].cumsum()
opp_data["3P_cum"] = opp_data["3P"].cumsum()
opp_data["FGA_cum"] = opp_data["FGA"].cumsum()
opp_data['TOV_cum'] = opp_data['TOV'].cumsum()
opp_data['FTA_cum'] = opp_data['FTA'].cumsum()
opp_data['ORB_cum'] = opp_data['ORB'].cumsum()
opp_data['TRB_cum'] = opp_data['TRB'].cumsum()
opp_data['FT_cum'] = opp_data['FT'].cumsum()
opp_data.head()

,Date,H/A,Opp,W/L,Team Score,Opp Score,FG,FGA,FG%,3P,...,TOV,PF,FG_cum,3P_cum,FGA_cum,TOV_cum,FTA_cum,ORB_cum,TRB_cum,FT_cum
G,,,,,,,,,,,,,,,,,,,,,
1.0,2017-10-18,vs,DEN,W,106.0,96.0,36.0,77.0,0.468,13.0,...,20.0,18.0,36.0,13.0,77.0,20.0,12.0,7.0,38.0,11.0
2.0,2017-10-20,@,MIN,L,97.0,100.0,36.0,81.0,0.444,7.0,...,15.0,17.0,72.0,20.0,158.0,35.0,38.0,17.0,82.0,32.0
3.0,2017-10-21,vs,OKC,W,96.0,87.0,32.0,78.0,0.410,11.0,...,17.0,16.0,104.0,31.0,236.0,52.0,57.0,25.0,120.0,44.0
4.0,2017-10-24,@,LAC,L,84.0,102.0,39.0,83.0,0.470,10.0,...,13.0,16.0,143.0,41.0,319.0,65.0,78.0,36.0,164.0,58.0
5.0,2017-10-25,@,PHO,L,88.0,97.0,42.0,96.0,0.438,6.0,...,16.0,25.0,185.0,47.0,415.0,81.0,87.0,52.0,219.0,65.0


### 2.3 Feature engineering
Obviously a good model would just look at total points scored. We had to engineer it.  Blah blah blah. Taking out the points from the box scores. Also talking about four factors from SOURCE.

In [51]:
team_four_factors = pd.DataFrame({'Date':[],
                                  'H/A':[],
                                  'eFG%':[],
                                  'TOV%':[],
                                  'ORB%':[],
                                  'FTR':[],
                                  'Score':[]})

team_four_factors['Date'] = team_data['Date']
team_four_factors['H/A'] = team_data['H/A']
team_four_factors["eFG%"] = (team_data['FG_cum'] + 0.5 * team_data['3P_cum']) / team_data['FGA_cum']
team_four_factors['TOV%'] = team_data['TOV_cum'] / (team_data['FGA_cum'] + 0.44 * team_data['FTA_cum'] + team_data['TOV_cum'])
team_four_factors['ORB%'] = team_data['ORB'] / (team_data['ORB_cum'] + (opp_data['TRB_cum'] - opp_data['ORB_cum']))
team_four_factors['FTR'] = team_data['FT_cum'] / team_data['FGA_cum']
team_four_factors['Score'] = team_data['Team Score']
team_four_factors.set_index('Date', inplace=True)
team_four_factors.loc[team_four_factors['H/A'] == '@', 'H/A'] = 0
team_four_factors.loc[team_four_factors['H/A'] == 'vs', 'H/A'] = 1
# team_four_factors.drop(team_four_factors['H/A'] == 0, 'H/A')
team_four_factors.head()

,H/A,eFG%,TOV%,ORB%,FTR,Score
Date,,,,,,
2017-10-18,1,0.561728,0.145575,0.162162,0.185185,106.0
2017-10-20,0,0.550633,0.164633,0.089744,0.183544,97.0
2017-10-21,1,0.551064,0.169007,0.060870,0.170213,96.0
2017-10-24,0,0.531847,0.170932,0.051282,0.156051,84.0
2017-10-25,0,0.516624,0.178667,0.034653,0.171355,88.0


In [39]:
opp_four_factors = pd.DataFrame({'Team':[],
                                 'Date':[],
                                 'eFG%':[],
                                 'TOV%':[],
                                 'ORB%':[],
                                 'FTR':[],})

opp_four_factors['Team'] = opp_data['Opp']
opp_four_factors['Date'] = opp_data['Date']
opp_four_factors['eFG%'] = (opp_data['FG_cum'] + 0.5 * opp_data['3P_cum']) / opp_data['FGA_cum']
opp_four_factors['TOV%'] = opp_data['TOV_cum'] / (opp_data['FGA_cum'] + 0.44 * opp_data['FTA_cum'] + opp_data['TOV_cum'])
opp_four_factors['ORB%'] = opp_data['ORB_cum'] / (opp_data['ORB_cum'] + (team_data['TRB_cum'] - team_data['ORB_cum']))
opp_four_factors['FTR'] = opp_data['FT_cum'] / opp_data['FGA_cum']
opp_four_factors['Score'] = opp_data['Opp Score']
opp_four_factors.set_index('Date', inplace=True)
opp_four_factors.head()

,Team,eFG%,TOV%,ORB%,FTR,Score
Date,,,,,,
2017-10-18,DEN,0.551948,0.195542,0.189189,0.142857,96.0
2017-10-20,MIN,0.518987,0.166889,0.217949,0.202532,100.0
2017-10-21,OKC,0.506356,0.166092,0.208333,0.186441,87.0
2017-10-24,LAC,0.512539,0.155383,0.219512,0.181818,102.0
2017-10-25,PHO,0.502410,0.151606,0.239631,0.156627,97.0


In [64]:
result = team_four_factors.join(opp_four_factors, rsuffix='_opp', on='Date')
result.head()

,H/A,eFG%,TOV%,ORB%,FTR,Score,Team,eFG%_opp,TOV%_opp,ORB%_opp,FTR_opp,Score_opp
Date,,,,,,,,,,,,
2017-10-18,1,0.561728,0.145575,0.162162,0.185185,106.0,DEN,0.551948,0.195542,0.189189,0.142857,96.0
2017-10-20,0,0.550633,0.164633,0.089744,0.183544,97.0,MIN,0.518987,0.166889,0.217949,0.202532,100.0
2017-10-21,1,0.551064,0.169007,0.060870,0.170213,96.0,OKC,0.506356,0.166092,0.208333,0.186441,87.0
2017-10-24,0,0.531847,0.170932,0.051282,0.156051,84.0,LAC,0.512539,0.155383,0.219512,0.181818,102.0
2017-10-25,0,0.516624,0.178667,0.034653,0.171355,88.0,PHO,0.502410,0.151606,0.239631,0.156627,97.0


In [65]:
result = result[result['H/A'] != 0]

In [68]:
result

,H/A,eFG%,TOV%,ORB%,FTR,Score,Team,eFG%_opp,TOV%_opp,ORB%_opp,FTR_opp,Score_opp
Date,,,,,,,,,,,,
2017-10-18,1,0.561728,0.145575,0.162162,0.185185,106.0,DEN,0.551948,0.195542,0.189189,0.142857,96.0
2017-10-21,1,0.551064,0.169007,0.060870,0.170213,96.0,OKC,0.506356,0.166092,0.208333,0.186441,87.0
2017-10-28,1,0.522340,0.182039,0.058091,0.161702,96.0,LAL,0.487903,0.151544,0.248120,0.159274,81.0
2017-10-30,1,0.529038,0.175863,0.021429,0.159710,104.0,DAL,0.493007,0.162434,0.246711,0.153846,89.0
2017-11-01,1,0.517941,0.167516,0.030120,0.185647,112.0,POR,0.486466,0.159206,0.238889,0.162406,103.0
2017-11-03,1,0.514563,0.162945,0.026596,0.195562,100.0,TOR,0.498641,0.159400,0.234536,0.176630,109.0
2017-11-07,1,0.495595,0.151362,0.028986,0.209251,97.0,PHI,0.519488,0.164352,0.227571,0.191537,104.0
2017-11-10,1,0.486253,0.152727,0.007576,0.212831,74.0,MIA,0.514359,0.165834,0.222000,0.190769,84.0
2017-11-11,1,0.491097,0.151939,0.021016,0.215558,114.0,BRK,0.513796,0.164932,0.223247,0.204567,106.0
